In [2]:
file = '/Users/alexandrasouly/code/chai/demo-voyager/frame_pairs/temp_test2/test.json'
import json
with open(file) as f:
    data = json.load(f)
for k,entry in data.items():
    entry["human_eval_similarity"] = ""
with open(file, 'w') as f:
    json.dump(data, f, indent=4)

 


In [8]:
file = '/Users/alexandrasouly/code/chai/demo-voyager/frame_pairs/temp_test2/test.json'
import json
with open(file) as f:
    data = json.load(f)

false_neg = 0
false_pos = 0
true_pos = 0
true_neg = 0
false_pos_list = []
false_neg_list = []
for k,entry in data.items():
    human = entry["human_eval_similarity"]
    agent = entry["similarity"]
    if agent not in ["Correct", "Incorrect", "Unknown"]:
        raise Exception(f"Agent similarity not in correct format:{k}")
    if human not in ["Correct", "Incorrect", "Unknown"]:
        raise Exception(f"Human similarity not in correct format:{k}")
    if human == "Correct":
        if agent == "Correct":
            true_pos += 1
        else:
            false_neg_list.append((k,entry))
            false_neg += 1
    elif human == "Incorrect":
        if agent == "Incorrect":
            true_neg += 1
        else:
            false_pos_list.append((k,entry))
            false_pos += 1
# print confusion matrix
print(f"True positives: {true_pos}, {true_pos/len(data)}")
print(f"True negatives: {true_neg}, {true_neg/len(data)}")
print(f"False positives: {false_pos}, {false_pos/len(data)}")
print(f"False negatives: {false_neg},  {false_neg/len(data)}")




True positives: 55, 0.4583333333333333
True negatives: 48, 0.4
False positives: 3, 0.025
False negatives: 14,  0.11666666666666667


In [10]:
# print false positives
print("False positives (agent thought correct, but not):")

for k,entry in false_pos_list:
    print(k,)
    print("Truths:")
    print(entry["ground_truths"])
    print("Preds:")
    print(entry["prediction"])

False positives (agent thought correct, but not):
/Users/alexandrasouly/code/chai/magical/frame_pairs/Task1/10-frame-pairs/pair2
Truths:
['Move the green triangle block B1 to the top left corner.', 'Move to top left with block B1', 'Push block B1 (green triangle) half way to the top left', 'Move block B1 to the top-left corner.']
Preds:
None of the above. The correct subgoal would be: Move R and B1 upwards towards the top of the arena.
/Users/alexandrasouly/code/chai/magical/frame_pairs/Task1/40-frame-pairs/pair9
Truths:
['Move the green square block B1 to the top left corner.', 'Move to bottom and pick up block B1 and move to top left with block B1', 'Grasp B1 and move it to the top left', 'Move block B1 to the top-left corner of the arena.']
Preds:
Move to the top of the arena while holding block B1.
/Users/alexandrasouly/code/chai/magical/frame_pairs/Task1/80-frame-pairs/pair1
Truths:
['Move the green square block B1 to the top left corner.', 'Move to bottom and pick up block B1 and

Why these mistakes happened:
- 1. it thought top is good enough for top left, because one of the ground truths said halfway to top left. Arguable, not a huge error.
- 2. okay same error, will need to specify this.
- 3. same error


- All of these can be fixed by specifying top left is not the same as top.


In [11]:
# print false negatives
print("False negatives (agent thought incorrect, but correct):")
for k,entry in false_neg_list:
    print(k,)
    print("Truths:")
    print(entry["ground_truths"])
    print("Preds:")
    print(entry["prediction"])

False negatives (agent thought incorrect, but correct):
/Users/alexandrasouly/code/chai/magical/frame_pairs/Task1/20-frame-pairs/pair1
Truths:
['Grab the green square block B1.', 'Move to bottom and place block B1', "Grasp B1 and then spin around so that you're facing upwards", 'Move to block B1 and pick it up.']
Preds:
Pick up B1.
/Users/alexandrasouly/code/chai/magical/frame_pairs/Task1/20-frame-pairs/pair6
Truths:
['Move the blue square block B1 towards the top left corner.', 'Move to top left with block B1', "Spin around to face up and a little bit to the left while grasping B1, then push B1 forward so that it's about two thirds of the way to the top of the arena", 'Move block B1 to the top-left corner.']
Preds:
Move B1 slightly upward within the central area of the arena.
/Users/alexandrasouly/code/chai/magical/frame_pairs/Task1/20-frame-pairs/pair9
Truths:
['Release the green square block B1 in the top left corner.', 'Move to top left with block B1', 'Release B1', 'Release block 

- 1. Didn't get that pick up was enough. Although one of the labels was wrong
- 2. Was actually hard to tell without looking at pic, fair enough
- 3. Maintain position vs release, kinda hard to tell fair enough
- 4. Doesn't understand that the robot is moving the block
- 5. Didint' get that picking up was enough
- 6. Doesn't understadn robot is doing the picking up
- 7. Didn't get that picking up was enough
- 8. fair enough
- 9. bad ground truth label was confusing
- 10. bad label was confusing
- 11. fair enough
- 12. didn't get that the agent is going towards a block to pick it up 
- 13. bad labels
- 14. bad labels

- Maybe specify grab and pick up are the same thing?
- And that if one of the ground truth says pick up only, then predicting pick up only is fine.
- specify that move R and B1 is the same as moving B1, because the R is moving it
- the robot is usually moving towards a block to pick it up
- Maybe go trough the subgoals and check if they match one by one. to fix the pick up issue.

This would fix 10/14 errors which is pretty good

# New prompts:

- get new prompt
- eval with it (careful not to overwrite stuff!!!!)
- add your predictions
- check how you are doing now, hopefully all good